<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import utilities
import os
import shutil
from subprocess import check_output

In [13]:
!nvidia-smi

Tue May  5 20:04:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    32W / 250W |   8745MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [0]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [5]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T20-02-49'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 62 (delta 0), reused 38 (delta 0), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [0]:
os.chdir(os.environ['PROJECT_DIR'])

In [7]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T20-02-49
total 54
-rw------- 1 root root 35149 May  5 20:02 LICENSE
drwx------ 2 root root  4096 May  5 20:02 notebooks
-rw------- 1 root root  6735 May  5 20:02 README.md
drwx------ 2 root root  4096 May  5 20:02 requirements
drwx------ 9 root root  4096 May  5 20:02 src


In [8]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0-rc4


In [9]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 1.4MB 7.8MB/s 
     |████████████████████████████████| 4.3MB 27.3MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 112kB 57.2MB/s 
     |████████████████████████████████| 460kB 59.9MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=4bfd64f33d38bfbd4762addc51fefc7b1fc0947bc417e11a6b240717ce4450ac
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=b2127264b79b2a9fa5f704e963feb8bc3dd9ae12d7e4f448e9339d953d66b1c9
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filenam

In [10]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T20-02-49/.env'

In [0]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [14]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'file_data_raw': '../data/interim/method-names-rich/elasticsearch.csv',
  'file_model_dir': 'models/saved/baseline/',
  'dir_preprocessed_data': '../data/interim/preprocessed/',
  'max_input_length': 200,
  'max_output_length': 8,
  'input_vocab_size': 20000,
  'input_embedding_dim': 128,
  'output_vocab_size': 6000,
  'output_embedding_dim': 128,
  'latent_dim': 512,
  'learning_rate': 0.0001,
  'epochs': 10,
  'batch_size': 512,
  'random_seed': 1,
}

run(Namespace(**params))

Experiment parameters:  Namespace(batch_size=512, dir_preprocessed_data='../data/interim/preprocessed/', epochs=10, file_data_raw='../data/interim/method-names-rich/elasticsearch.csv', file_model_dir='models/saved/baseline/', input_embedding_dim=128, input_vocab_size=20000, latent_dim=512, learning_rate=0.0001, max_input_length=200, max_output_length=8, output_embedding_dim=128, output_vocab_size=6000, random_seed=1)


Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
epoch 1 - batch 10 - loss 3.6819539070129395 - f1_macro_score 7.583904516650364e-05 - sparse_categorical_accuracy 0.12176339328289032
epoch 1 - batch 20 - loss 3.461566209793091 - f1_macro_score 0.00011424954573158175 - sparse_categorical_accuracy 0.15831473469734192
epoch 1 - batch 30 - loss 3.2588179111480713 - f1_macro_score 0.00013194359780754894 - sparse_categorical_accuracy 0.17552083730697632
epoch 1 - batch 40 - loss 2.5391979217529297 - f1_macro_score 0.00014980611740611494 - sparse_categorical_accuracy 0.1748046875
epoch 1 - batch 50 - loss 2.1294023990631104 - f1_macro_score 0.00015833365614525974 - sparse_categorical_accuracy 0.17184151709079742
epoch 1 - batch 60 - loss 2.3676512241363525 - f1_macro_score 0.00017253927944693714 - sparse_categorical_accuracy 0.17052176594734192
epoch 1 - batch 70 - loss 2.08333683013916 - f1_macro_score 0.00017588352784514427 - sparse

KeyboardInterrupt: ignored

Error in callback <function _init_jupyter.<locals>.cleanup at 0x7f8d7a261b70> (for post_run_cell):


KeyboardInterrupt: ignored